In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

# Fourier Ptychographic Imaging

## 1. Forward Imaging Model

In this process, the sample is illuminated with one LED of the LED illuminator at time and a camera captures Lo-Res images under different incident angles. To simulate it, we have to:

1. Create a Hi-Res complex image.

2. Generate the incident wave vectors.

3. Produce the ouput Lo-Res images.

#####  [2. The Recovery Process](phaseretrieval.ipynb)

In [ ]:
from PIL import Image
import numpy as np
import math
import matplotlib.pyplot as plt
import decimal
import scipy.misc
import os

In [ ]:
%matplotlib inline

#### 1.1. Create the Hi-Res complex image.

In [ ]:
def generate_obj(amplitude, phase, **kwargs):
    """Generates the object to be photographed and returns it as a
    numpy.ndarray.
    
    The object is a Hi-Res complex image."""
    show = kwargs.pop('show', False)  # Do I show the image in the
                                      # current notebook?
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_obj.__name__,
                  list(kwargs.keys())[-1]))
    amplitude = Image.open(amplitude)
    phase = Image.open(phase)
    w, h = amplitude.size
    # Some programming languages such as MATLAB, the resize methods
    # use bicubic interpolation by default.
    phase = phase.resize((w, h), resample=Image.BICUBIC)
    arr_ampl = np.array(amplitude,
                        dtype='d')  # 'd' (str) for a double-precision
                                    # floating-point number. 
    arr_phase = np.array(phase, dtype='d')
    amplitude.close()
    phase.close()
    arr_phase = math.pi * arr_phase / np.amax(arr_phase)
    obj = arr_ampl * np.exp(1j * arr_phase)
    obj = np.absolute(obj)
    if show:
        # Using PIL, the objects are shown as images almost completely
        # white.
        #
        # Using matplotlib anc 'Greys_r' (str) as value for cmap key
        # [1], the object is shown such as in Matlab R2009b.
        #
        # [1] unutbu. (2010). Display image as grayscale using
        # matplotlib [Msg 1]. Message posted to
        # http://stackoverflow.com/questions/3823752/display-image-as-grayscale-using-matplotlib?answertab=votes#tab-top
        plt_img = plt.imshow(obj, cmap='Greys_r')
    return obj

#### 1.2. Generate the incident wave vectors.

In [ ]:
def gen_xy_led_ring(ledsperring, d):
    """Returns a partitioned list with the (x, y) coordinates of each
    LED of ledsperring (list) spaced d (float) mm from another
    neighboring in a LED ring."""
    l_xy = list()
    l_row = list()
    l = list()
    for nleds in ledsperring:
        r = (d * nleds) / (2.0 * math.pi)
        theta_step = 360 // nleds  # For the next line, theta_step
                                   # must be str.
        range_theta = range(0, 360, theta_step)
        range_theta = range_theta[:nleds]
        for theta in range_theta:
            x = r * math.cos(math.radians(theta))
            y = r * math.sin(math.radians(theta))
            l_xy.append((x, y))
    order = math.ceil(math.sqrt(len(l_xy)))
    for i in range(order):
        for j in range(order):
            l_row.append((0.0, 0.0))
        l.append(l_row)
        l_row = list()
    len_lxy = len(l_xy)
    for i in range(order):
        for j in range(order):
            idx = i * (order + 1) + j
            if idx < len(l_xy):
                l[i][j] = l_xy[idx]
    return l

In [ ]:
def gen_xy_led_grid(n, d):
    """Returns a list with the (x, y) coordinates of each LED spaced d
    (int) mm from another in a n-by-n LED grid."""
    l_xy = list()
    x_max = math.floor((n / 2)) * d
    x_min = -x_max
    y_max = x_max
    y_min = x_min
    l_range = list(range(x_min, x_max + 1, int(d)))
    l_row = list()
    for i in l_range:
        for j in l_range:
            l_row.append((j, -i))  # (x, y)
        l_xy.append(l_row)
        l_row = list()
    return l_xy

In [ ]:
def generate_wave_vectors(l_xy, height):
    """Returns a numpy.ndarray with tuples with magnitudes of the
    components of the incident wave vectors that emerge from a LED
    illuminator spaced height (float) mm from the sample. The location
    of each LED in the illuminator is a tuple in l_xy (list)."""
    arr = np.array(l_xy)
    arr = arr / height
    arr = np.arctan(arr)
    arr = np.sin(arr)
    return -arr

In [ ]:
def round_half_up(num):
    """In Python 3, the round() function had changed from Python 2.
    For example, in Python 3, round(2.5) returns 2 ('int') such
    round(1.5). It is possible to obtain 3 ('int') instead 2 using
    the decimal module [2].
    
    [2] Barthelemy. (2014). Python 3.x rounding behavior. Message
    posted to
    http://stackoverflow.com/questions/10825926/python-3-x-rounding-behavior
    """
    return int(decimal.Decimal(num).quantize(decimal.Decimal(1),
                                             rounding=decimal.ROUND_HALF_UP))

In [ ]:
def get_cft(wave_vectors, wavelen, ccdpx, na, w, h):
    """Generates the coherent transfer function of the coherent
    imaging system.
    
    Args:
        wave_vectors: A numpy.ndarray with components (tuples) of wave
            vectors.
        wavelen: A wavelength in mm.
        ccdpx: A sampling pixel size of the CCD (float).
        na: A numerical aperture (float) of the objective lens.
        w: The width in pixels (int) of the output Hi-Res output
            image.
        h: The height (int) of the output Hi-Res output.
    
    Returns:
        p: The height of the output in pixels (int).
        q: The width of the output (int).
        dkx: A float number. See its formula in its assignment line.
        dky: A float number. See its formula in its assignment line.
        kx: A numpy.ndarray with x-components of k_0 * wave_vectors.
        ky: A numpy.ndarray with y-components of k_0 * wave_vctors.
        cft: The coherent transfer function (numpy.ndarray).
    """
    k_0 = 2 * math.pi / wavelen
    hirespx = ccdpx / 4  # Pixel size of the reconstruction.
    dkx = 2 * math.pi / (hirespx * w)
    dky = 2 * math.pi / (hirespx * h)
    p = int(h / (ccdpx / hirespx))  
    q = int(w / (ccdpx / hirespx))  # Number of columns of the output.
    k = k_0 * wave_vectors;
    # From all rows, all columns, extract the first element.
    kx = k[:, :, 0]
    # Reshape xk ('numpy.ndarray') in 1-D array.
    kx = np.reshape(kx, len(kx.flat))
    ky = k[:, :, 1]
    ky = np.reshape(ky, len(ky.flat))
    cutoff_freq = na * k_0
    k_max = math.pi / ccdpx
    kxm, kym = np.meshgrid(np.arange(-k_max, k_max + 1,
                                     k_max / ((q - 1) / 2)),
                           np.arange(-k_max, k_max + 1,
                                     k_max / ((p - 1) / 2)))
    cft = ((kxm ** 2 + kym ** 2) < cutoff_freq ** 2)
    cft = cft.astype(float)
    return p, q, dkx, dky, kx, ky, cft

#### 3. Produce the ouput Lo-Res images.

In [ ]:
def num_str_zeros(num, n_digs, firstis1=False):
    """Returns a string that contains a sequence of n_digs - len(num)
    zeros followed by num (int).
    
    Args:
        num: A non-negative integer number.
        n_digs: The length of the string that will contains zeros and
            num.
        firstis1: Plus 1 to num if it is True. It is useful for
            filenames.
    
    >>> num_str_zeros(89, 5)
    '00089'
    >>> num_str_zeros(0, 4, True)
    '0001'
    """
    if num < 0:
        num = 0
    if firstis1:
        num += 1
    len_num = len(str(int(num)))
    str_num = ''
    for i in range(n_digs - len_num):
        str_num += '0'
    str_num += str(int(num))
    return str_num

In [ ]:
def find_out_dir(**kwargs):
    """Find out a directory and returns the route.
    
    With replace (bool) equal to False, this will create a new
    directory if dirname (string) exists in <s3out>/<parentdir>/
    
    From blendjupyter.ipynb (Blender kernel)."""
    s3out = kwargs.pop('s3out',
                       os.path.join(os.path.expanduser('~'),
                                    's3-out'))
    dirname = kwargs.pop('dirname', 'blend-phg-set-0001')
    parentdir = kwargs.pop('parentdir', 'scanner')
    path = kwargs.pop('path', os.path.join(s3out, parentdir, dirname))
    replace = kwargs.pop('replace', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(find_out_dir.__name__,
                  list(kwargs.keys())[-1]))
    if not replace:
        # Create a new direcctory with a different name.
        while True:
            if not os.path.exists(path):
                print('Make directory:', path)
                os.makedirs(path)
                break
            else:
                str_num = path.split('-')[-1]
                num_dir = int(str_num)
                num_dir += 1
                str_num_dir = num_str_zeros(num_dir, len(str_num))
                path = ('-'.join(path.split('-')[:-1]) + '-'
                        + str_num_dir)
    return path

In [ ]:
def generate_lores_set(obj, cft, leds, kx, ky, w, h, dkx, dky, p, q,
                       outpath, **kwargs):
    """Generates a low-resolution photo set and returns it as a
    numpy.ndarray in which each photo is immediately taken after a LED
    lights the sample.
    
    Each image only contains the amplitude information.
    
    Args:
        obj: A Hi-Res complex input image.
        cft: The coherent transfer function (numpy.ndarray).
        leds: A number of LEDs used during the photo capture process.
        kx: A numpy.ndarray with x-components of k_0 * wave_vectors.
            See get_cft().
        ky: A numpy.ndarray with y-components of k_0 * wave_vctors.
            See get_cft().
        w: The width in pixels (int) of the output Hi-Res output
            image.
        h: The height (int) of the output Hi-Res output.
        dkx: A float number. See get_cft().
        dky: A float number. See get_cft().
        p: The height of the output in pixels (int).
        q: The width of the output (int).
        outpath: A route for the output Lo-Res images.
        **kwargs: Keyword arguments. See inline comments.
    """
    prefix = kwargs.pop('prefix', 'lores-img_')  # Prefix in the name
                                                 # of the output file.
    extension = kwargs.pop('extension', '.tif')  # Extension of the
                                                 # output file.
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_lores_set.__name__,
                  list(kwargs.keys())[-1]))
    obj_ft = np.fft.fftshift(np.fft.fft2(obj))
    lores_imgs_seq = list()  # The Lo-Res images sequence.
    for i in range(leds):
        kxc = round_half_up((w + 1) / 2.0 + kx[i] / dkx)
        kyc = round_half_up((h + 1) / 2.0 + ky[i] / dky)
        kyl = round_half_up(kyc - (p - 1) / 2.0);
        kyh = round_half_up(kyc + (p - 1) / 2.0);
        kxl = round_half_up(kxc - (q - 1) / 2.0);
        kxh = round_half_up(kxc + (q - 1) / 2.0);
        lores_imgs_seq_ft = (p / h) ** 2
        lores_imgs_seq_ft *= obj_ft[kyl - 1:kyh, kxl - 1:kxh]
        lores_imgs_seq_ft *= cft
        img_lores = np.fft.ifft2(np.fft.ifftshift(lores_imgs_seq_ft))
        img_lores = np.absolute(img_lores)
        lores_imgs_seq.append(img_lores)
        str_i = num_str_zeros(i, len(str(leds)), True)
        filename = prefix + str_i + extension
        img_path = os.path.join(outpath, filename)
        # scipy.misc.imsave() rescales the dynamic range of the pixel
        # values [3]. Add cmin and cmax as parameters to
        # scipy.misc.toimage() as follow to prevent the rescaling.
        # 
        # [3] Lippens, S. (2016). Saving of images in scipy and
        # preventing dynamic range rescaling.
        scipy.misc.toimage(img_lores, cmin=0, cmax=255).save(img_path)
        print('Saved Lo-Res image:', img_path)
    return np.array(lores_imgs_seq)

In [ ]:
def simulate_set(leds, **kwargs):
    """Carries out all steps of capturing photos process in the
    Fourier Ptychography algorithm and returns the images as a
    three-dimensional numpy.ndarray and the route of the folder
    that contains them.
    
    Args:
        leds: A number of LEDs (int) per row or column of a squared
            LED illuminator or a list of positive integer numbers that
            represent the number of LEDs per ring from the center to
            the edge in a LED ring illuminator.
        **kwargs: Keyword arguments. See inline comments.
    """
    amp_pics = {
        'wing': 'BobMacInnes_microscopy-043--fly-wing_flickr_1024x685.tif',
        'flake': 'Jasper-Nance_flake1_flickr_3008x2008.tif',
        'caterpillar': 'Macroscopic-Solutions_Crambidia-myrolosea_flickr_2048x1038.tif',
         }  # Recommended pictures (CC) inside of fp-imgs-cc folder
            # to be used as amplitude during Fourier Ptychography
            # process. REMEMBER UNCOMPRESS fp-imgs-cc.zip
    ph_pics = {
        'valpo': 'pabloaez_Valparaiso-054_flickr_4000x3000.png'
        }  # "Recommended pictures (CC) [..] to be used [...]
           # [as phase]".
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'superscanner-software-s3'))
    
    outpath = kwargs.pop('outpath', None)
    if outpath is None:
        outpath = find_out_dir(dirname='lores-set-0001',
                               parentdir='microscope')
    ext = kwargs.pop('ext', '.tif')  # Extension of the output files.
    # Does the first image have a 1 at the end of its filename?
    namefrom1 = kwargs.pop('namefrom1', True)
    wavelen = kwargs.pop('wavelen', 0.63e-6)  # In mm.
    ccdpx = kwargs.pop('ccdpx', 2.75e-6)  # Sampling pixel size of the
                                          # CCD.
    na = kwargs.pop('na', 0.08)  # Numerical aperture of the employed
                                 # objective lens.
    d = kwargs.pop('d', 4)  # Distance in mm between neighboring LEDs.
    h = kwargs.pop('h', 90)  # Distance in mm between the LED
                             # illuminator and the sample.
    amplitude = kwargs.pop('amplitude', 'flake')
    phase = kwargs.pop('phase', 'valpo')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(simulate_set.__name__,
                  list(kwargs.keys())[-1]))
    if amplitude in amp_pics.keys():
        amplitude = os.path.join(s3path, 'img', 'fp-imgs-cc',
                                 amp_pics[amplitude])
    if phase in ph_pics.keys():
        phase = os.path.join(s3path, 'img', 'fp-imgs-cc',
                            ph_pics[phase])
    l_xy_leds = list()  # List for the (x, y) coordinates of the LED
                        # illuminator.
    if isinstance(leds, int):
        l_xy_leds = gen_xy_led_grid(leds, d)
        n_leds = leds ** 2
    elif all(isinstance(led, int) and led > 0 for led in leds):
        # leds is a list of integer numbers.
        l_xy_leds = gen_xy_led_ring(leds, d)
        n_leds = sum(leds)
    wvs = generate_wave_vectors(l_xy_leds, h)
    in_obj = generate_obj(amplitude, phase)
    hpx, wpx = in_obj.shape
    p, q, dkx, dky, kx, ky, cft = get_cft(wvs, wavelen, ccdpx, na,
                                          wpx, hpx)
    lores_imgs_seq = generate_lores_set(in_obj, cft, n_leds, kx, ky,
                                        wpx, hpx, dkx, dky, p, q,
                                        outpath, extension=ext)
    return lores_imgs_seq, outpath